![](../img/results-7.png)

## Load Data

In [1]:
import re
import time
import sys
import warnings
import os

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import sklearn
from IPython.core.display_functions import display
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import RandomizedSearchCV
from sklearn.linear_model import SGDClassifier
from sklearn.utils.fixes import loguniform
from sklearn.utils import shuffle

dataframe = pd.read_csv('../URL_Classification.csv', header=None)
dataframe.columns = ["index", "url", "label"]

## Preprocessing

In [2]:
dataframe['url'] = dataframe['url'].apply(lambda x: np.str_(x))

## Prepare labels

In [3]:
labels = set(dataframe['label'])
label2id = {l: n for n, l in enumerate(labels)}
dataframe['label'] = dataframe['label'].map(label2id)

## Transform with count vectorizer

In [4]:
count_vectorizer = CountVectorizer(analyzer='char_wb', ngram_range=(5, 5))
X = count_vectorizer.fit_transform(dataframe['url'])
y = dataframe['label'].values

X, y = shuffle(X, y)

## Optimize alpha parameter with RandomizedSearchCV

In [5]:
if not sys.warnoptions:
    warnings.simplefilter("ignore")
    os.environ["PYTHONWARNINGS"] = "ignore" # Also affect subprocesses


sgd = sklearn.linear_model.SGDClassifier(class_weight='balanced', loss='hinge')

dist = {'alpha': loguniform(1e-7, 1e-1)}
clf = RandomizedSearchCV(sgd, param_distributions=dist, random_state=67, n_jobs=5, n_iter=150)

start = time.time()
search = clf.fit(X, y)
print(f'\nCV Time: {time.time()-start}')
search.best_params_


CV Time: 26771.25333404541


{'alpha': 3.495077991813885e-06}

## Report results

In [6]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    display(pd.DataFrame(clf.cv_results_))

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_alpha,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,95.062697,0.508160,0.775813,0.051785,0.000188,{'alpha': 0.00018841200866151887},0.552729,0.553542,0.539556,0.554174,0.529452,0.545891,0.009842,85
1,71.999986,0.468458,0.725011,0.108517,0.014228,{'alpha': 0.014227862634159776},0.468771,0.476238,0.469321,0.467279,0.470263,0.470375,0.003087,130
2,85.855476,0.271478,0.857922,0.089093,0.001304,{'alpha': 0.0013042340053544472},0.508177,0.506360,0.505157,0.506425,0.509512,0.507126,0.001533,103
3,174.592781,2.155331,0.791256,0.057566,0.00001,{'alpha': 9.762272497212736e-06},0.570913,0.585084,0.587135,0.585835,0.582533,0.582300,0.005888,43
4,405.247056,4.018027,0.834997,0.047908,0.0,{'alpha': 2.2908021253208054e-07},0.577826,0.576981,0.578654,0.576497,0.576292,0.577250,0.000879,54
5,143.459175,1.494576,0.700005,0.062020,0.000021,{'alpha': 2.078101167061151e-05},0.586201,0.575500,0.577365,0.586203,0.576231,0.580300,0.004855,48
6,256.551536,3.155801,0.816198,0.054470,0.000002,{'alpha': 1.9006368801574431e-06},0.585878,0.601674,0.591757,0.592953,0.583330,0.591118,0.006377,23
7,74.589652,0.675859,0.797704,0.053588,0.039359,{'alpha': 0.03935862342423758},0.458525,0.456868,0.458422,0.454521,0.450999,0.455867,0.002832,148
8,85.362997,0.290119,0.768760,0.069465,0.002172,{'alpha': 0.00217224993617838},0.499571,0.500493,0.489491,0.499733,0.493092,0.496476,0.004395,109
9,422.913471,2.837907,0.785324,0.038353,0.0,{'alpha': 1.911489162287619e-07},0.568571,0.573289,0.568993,0.574878,0.573125,0.571771,0.002520,64


In [7]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    display(pd.DataFrame(clf.cv_results_).sort_values('rank_test_score'))

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_alpha,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
16,221.688238,1.352840,0.769872,0.057439,0.000003,{'alpha': 3.495077991813885e-06},0.604656,0.607957,0.593712,0.597534,0.600339,0.600840,0.005044,1
24,238.889447,1.329765,0.763134,0.042669,0.000002,{'alpha': 2.4938910461383064e-06},0.600001,0.601780,0.606236,0.597284,0.592818,0.599624,0.004479,2
105,201.797250,2.312472,0.739241,0.025024,0.000005,{'alpha': 5.4530449168258554e-06},0.598843,0.599749,0.606511,0.591763,0.595086,0.598390,0.004957,3
139,232.598924,1.958237,0.768748,0.026864,0.000003,{'alpha': 2.8702333958770317e-06},0.598181,0.597720,0.599553,0.596356,0.597815,0.597925,0.001023,4
57,237.033462,1.879183,0.779471,0.083629,0.000003,{'alpha': 2.7467787855796015e-06},0.598888,0.599118,0.596342,0.598928,0.593922,0.597440,0.002035,5
33,194.326194,1.456040,0.765626,0.022107,0.000006,{'alpha': 6.107326737919852e-06},0.598075,0.605260,0.586984,0.595573,0.600493,0.597277,0.006059,6
84,235.903080,0.596298,0.813675,0.052737,0.000003,{'alpha': 2.681750245958141e-06},0.600740,0.596310,0.596614,0.591721,0.600694,0.597216,0.003344,7
146,169.423537,1.223019,0.716202,0.055096,0.00001,{'alpha': 1.0480099931173604e-05},0.603795,0.598370,0.586789,0.593845,0.598180,0.596196,0.005664,8
137,213.737783,0.925648,0.812521,0.055017,0.000004,{'alpha': 4.149991063166486e-06},0.595152,0.597477,0.592055,0.604459,0.590656,0.595960,0.004870,9
77,237.797350,1.756830,0.757086,0.050924,0.000003,{'alpha': 2.6660403749505833e-06},0.604803,0.594755,0.599294,0.590844,0.587860,0.595511,0.006026,10
